# Speech to text를 사용한 오디오 세그먼트

비디오의 오디오 부분은 비디오의 구조와 내러티브에 대한 수많은 단서를 제공합니다. 오디오에는 다음이 포함됩니다:

* 단어, 문장부호, 문장 또는 자막과 같은 다른 언어 세그먼트로 더 세분화될 수 있는 Speech
* 스포츠 이벤트의 군중 소음, foley effects 또는 배경 음악과 같은 소리
* 무음을 포함한 사운드 레벨

아래 그림 1은 비디오의 서로 다른 시점의 사운드 레벨에 해당하는 오디오 파형이 있는 Meridian의 클립을 보여줍니다.

![Video with audio waveform](./static/images/01-meridian-audio-wide.png)
**그림 1. 오디오 파형이 있는 비디오**

이 노트북에서는 비디오의 speech에 초점을 맞춰 오디오를 사용하여 비디오를 세그먼트로 분해하는 기술을 계속 탐구할 것입니다. 구체적으로 다음을 수행합니다:

* Amazon Transcribe를 사용하여 오디오 대본을 추출하고 단어, 문장, 자막 및 speech 세그먼트를 식별
* 서로 다른 대화 주제를 포함하는 비디오 세그먼트를 식별하기 위해 Foundation Model 사용
* 비디오의 chapters를 식별하기 위해 대화 주제와 시각적 scenes 결합

이 노트북의 출력은 이후 워크숍의 사용 사례 섹션에서 사용될 것입니다. 예를 들어, Ad Break Detection and Contextual Ad Placement, Video Summarization 등에서 사용됩니다.

### 주요 용어 및 정의

노트북에서 사용되는 용어의 정의를 확인하고 싶을 때 이 섹션을 참조할 수 있습니다.

- **Transcript** - 비디오의 오디오 트랙에서 speech 내용을 나타내는 연속적인 단어와 문장부호의 시퀀스
- **Subtitle** - 시청자가 비디오를 시청할 때 표시되도록 의도된 비디오의 speech를 나타내는 텍스트 세그먼트
- **Speech Segment** - Amazon Transcribe의 출력인 speech 세그먼트
- **Conversation topic (aka topic)** - 유사한 주제에 대한 논의를 포함하는 문장 그룹의 요약
- **WebVTT** - 웹의 비디오 콘텐츠에 대한 자막과 같은 시간이 지정된 텍스트 트랙 데이터를 저장하는 데 사용되는 파일 형식


### 워크플로우

이 실습의 목적은 speech to text를 사용하여 비디오의 오디오 요소를 다루는 실습과 오디오에서 파생된 시간이 지정된 텍스트로 prompt engineering을 연습하는 것입니다. SageMaker Studio에서 AWS 서비스 API를 실행하게 됩니다. 워크플로우는 비디오를 입력으로 받아 transcript, sentence segments, WebVTT 형식 subtitles, conversation topics 출력을 생성합니다.

![Audio Workflow](./static/images/01-audio-workflow.png)

<div class="alert alert-block alert-info">
💡 이 Jupyter 노트북의 왼쪽 탐색 패널에서 목록 아이콘을 클릭하면 노트북의 개요와 현재 위치를 볼 수 있습니다.
</div>

## Prerequisites

### Import python packages

In [ ]:
from pathlib import Path
import os
import json
import boto3
import json_repair
from termcolor import colored
from IPython.display import JSON
from IPython.display import Video
from IPython.display import Pretty
from IPython.display import Image as DisplayImage
from lib.frames import VideoFrames
from lib.shots import Shots
from lib.scenes import Scenes
from lib.transcript import Transcript
from lib import util
import requests
from pathlib import Path
from tqdm import tqdm
from typing import Tuple
import time


### 이전 노트북에서 저장된 값 검색하기

이 노트북을 실행하려면 이전 노트북 [01A-visual-segments-frames-shots-scenes.ipynb](01A-visual-segments-frames-shots-scenes.ipynb)을 실행했어야 합니다.

In [ ]:
# Get variables from the previous notebook
%store -r

In [ ]:
%store


# Amazon Transcribe를 사용하여 transcript, 오디오 세그먼트 및 WebVTT 자막 생성하기

## Transcript 라이브러리

<div class="alert alert-block alert-info">
<b>참고</b>: Transcript 라이브러리를 위한 코드는 더 자세히 살펴보고 싶다면 이 프로젝트의 <b>lib</b> 폴더에 있지만, 이 연습의 목적은 transcript의 개념을 이해하는 것입니다.
</div>

[lib/transcript.py](lib/transcript.py)를 열려면 링크를 클릭합니다.

## Transcript 및 기타 파생 출력 생성

In [ ]:
video['transcript'] = Transcript(video["path"], session['bucket'])

## Amazon Transcribe 결과 검토
Amazon Transcribe의 응답에는 텍스트 전용 transcript와 각 단어 및 문장부호의 신뢰도 점수와 타임스탬프가 포함된 items 컬렉션이 포함된 transcript가 있는 results 딕셔너리가 포함되어 있습니다. 응답에는 또한 WebVTT 또는 SRT 형식의 자막으로 포맷된 동일한 transcript가 포함되어 있습니다. 이러한 출력들을 살펴보겠습니다.


##### Transcript JSON 출력

transcript의 `results` 속성에는 몇 가지 흥미롭고 유용한 출력이 포함되어 있습니다:
* **transcripts** - 비디오에 대한 대체 텍스트 전용 transcripts 목록입니다. 우리의 결과는 1개의 대안만 생성하도록 구성되어 있지만, 필요한 경우 Amazon Transcribe를 구성하여 더 많이 생성할 수 있습니다. 대안은 비디오의 speech에 대한 서로 다른 의미론적 해석일 뿐입니다.
* **items** - items는 Amazon Transcribe가 비디오의 speech에서 추론한 `pronunciations`(단어)와 `punctuation`의 시계열입니다. 이는 AI 추론이기 때문에 각 item에 대한 _confidence score_가 있습니다. Amazon Transcribe에서 confidence scores는 서비스가 각 전사된 단어의 정확성에 대해 얼마나 확신하는지를 나타냅니다. 마지막으로, 각 item에 대한 시작 및 종료 시간이 있어 items의 타이밍을 비디오의 다른 타임스탬프가 있는 요소와 정렬하는 데 사용할 수 있습니다.
* **audio_segments** - audio segments에는 Amazon Transcribe가 감지한 distinct speech segments 목록이 포함되어 있습니다. 세그먼트 경계는 다음에 의해 결정됩니다:
    * 자연스러운 speech 일시 중지
    * 화자 변경
    * 최대 세그먼트 지속 시간 제한
    * 문장부호

<div class="alert alert-block alert-info">
    ❓ <b>Amazon Q Developer에게 질문하기</b>: What are the attributes of items returned from Amazon Transcribe? 
<br></br>
    ❓ <b>Amazon Q Developer에게 질문하기</b>: What are confidence scores in Amazon Transcribe? 
</div>

아래 샘플 비디오의 `results`에서 이러한 각 속성을 잠시 살펴봅니다.

In [ ]:
JSON(filename=video['transcript'].transcript_file)

### WebVTT 자막

Amazon Transcribe에서 생성한 WebVTT 형식 자막의 처음 몇 줄을 살펴보겠습니다. WebVTT 자막은 비디오 플레이어에서 비디오의 speech를 화면에 텍스트로 표시하는 데 사용할 수 있습니다.

In [ ]:
!echo "$(<{video['transcript'].vtt_file})"

### 비디오와 함께 자막 재생하기

마지막으로, 생성된 자막 트랙과 함께 비디오를 보겠습니다. 참고: 비디오에서 첫 번째 speech가 발생하는 shot에서 비디오를 시작하기 위해 shot 정보를 사용했습니다.


In [ ]:
# Play the video with subtitles

from IPython.display import HTML
from base64 import b64encode
start = video['shots'].shots[8]['start_ms']/1000
end = video['shots'].shots[8]['end_ms']/1000
speech_shot_url = f'{video["url"]}#t={start}'

video_html = f"""
<video width="640" height="360" controls>
    <source src="{speech_shot_url}" type="video/mp4">
    <track src="{video['transcript'].vtt_file}" kind="captions" srclang="en" label="English" default>
</video>
"""

HTML(video_html)

# Amazon Bedrock을 사용하여 대화 주제 생성

다음 섹션에서는 generative AI를 사용하여 비디오 transcript에서 시간에 따라 발생하는 대화 주제를 이해할 것입니다. 이는 여러 Foundation Models로 수행할 수 있는 텍스트 요약 작업입니다.

<div class="alert alert-block alert-info">
💡 이 워크숍 전반에 걸쳐 이 작업과 다른 모든 generative AI 작업에 Amazon Bedrock의 Anthropic Claude Sonnet 3.5를 사용할 것입니다. 멀티모달(이미지와 텍스트) 입력에 대해 다양한 작업을 수행할 수 있는 유연성 때문에 Anthropic Claude Sonnet 3를 선택했습니다. 실제로는 요구 사항에 따라 다른 작업에 다른 FM을 대체할 수 있습니다. 예를 들어, 특정 사용 사례에서 Anthropic Claude Haiku가 더 낮은 가격으로 적절한 결과를 제공할 수 있습니다.
</div>

transcript를 Amazon Bedrock의 Anthropic Claude 3 Sonnet 모델에 전달할 것입니다. 모델은 transcript를 분석하고 특정 JSON 형식으로 대화 주제 포인트를 제안합니다. 프롬프트에서 각 주제에 주제를 설명하는 이유와 함께 시작 및 종료 타임스탬프가 포함되어야 함을 지정합니다. Sonnet 모델의 프롬프트는 아래와 같습니다. 이 프롬프트는 [Anthropic Claude Messages API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html)를 사용합니다:


**System prompt**

```
당신은 WebVTT 형식의 영화 대본을 분석하고 대화의 주제 변화를 기반으로 주제 포인트를 제안하는 미디어 운영 어시스턴트입니다. 전체 대본을 읽는 것이 중요합니다.
```


**Messages**

```
[
    {
        'content': '여기 <transcript> 태그 안에 대본이 있습니다:\n'
                '<transcript>{transcript}\n</transcript>\n',
        'role': 'user'
    },
    {
        'content': '네. 대본을 받았습니다. 어떤 출력 형식을 원하시나요?',
        'role': 'assistant'
    },
    {
        'content': 'JSON 형식입니다. 출력 예시:\n'
                '{"topics": [{"start": "00:00:10.000", "end": "00:00:32.000", '
                '"reason": "주제가 ...에 대해 이야기하는 것으로 보입니다"}]}\n',
        'role': 'user'
    },
    {
        'content': '{', 'role': 'assistant'
    }
 ]
```




<div class="alert alert-block alert-info">
    ❓ <b>Amazon Q Developer에게 질문하기</b>: What are the inputs for the Anthropic Claude Messages API?
    <br></br>* Jupyter 노트북의 사이드바 메뉴에서 Amazon Q Developer 메뉴 아이콘을 보셨을 것입니다. 노트북의 코드나 AWS API에 대해 질문이 있을 때마다 Amazon Q에게 물어보세요.
</div>

#### 프롬프트 구성하기

아래 코드는 Amazon Bedrock용 프롬프트를 구성한 다음 프롬프트를 실행하기 위해 Amazon Bedrock API를 호출합니다.

In [ ]:
from lib import bedrock_helper as brh
from lib import util

# MODEL_ID = 'anthropic.claude-3-5-sonnet-20240620-v1:0'
# MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"
MODEL_VER = 'bedrock-2023-05-31'
CLAUDE_PRICING = (0.00025, 0.00125)

def analyze_conversations(vtt_file):

    response = {}
    messages = []

    # transcript
    transcript_message = make_transcript(vtt_file)
    messages.append(transcript_message)

    # output format?
    messages.append({
        'role': 'assistant',
        'content': 'Got the transcript. What output format?'
    })

    # example output
    example_message = make_conversation_example()
    messages.append(example_message)

    # prefill output
    messages.append({
        'role': 'assistant',
        'content': '{'
    })

    ## system prompt to role play
    system = '''
    You are a media operation assistant who analyses movie transcripts in WebVTT format
    and suggest topic points based on the topic changes in the conversations. 
    It is important to read the entire transcript.
    '''

    ## setting up the model params
    model_params = {
        'anthropic_version': MODEL_VER,
        'max_tokens': 4096,
        'temperature': 0.1,
        'top_p': 0.7,
        'top_k': 20,
        'stop_sequences': ['\n\nHuman:'],
        'system': system,
        'messages': messages
    }

    response['model_params'] = model_params
    try:
        response['response'] = inference(model_params)
    except Exception as e:
        print(colored(f"ERR: inference: {str(e)}\n RETRY...", 'red'))
        response['response'] = inference(model_params)
    return response



def make_conversation_example():
    example = {
        'topics': [
            {
                'start': '00:00:10.000',
                'end': '00:00:32.000',
                'reason': 'It appears the topic talks about...'
            }
        ]
    }

    return {
        'role': 'user',
        'content': 'JSON format. An example of the output:\n{0}\n'.format(json.dumps(example))
    }

def make_transcript(vtt_file):
    with open(vtt_file, encoding="utf-8") as f:
        transcript = f.read()
    
    return {
        'role': 'user',
        'content': 'Here is the transcripts in <transcript> tag:\n<transcript>{0}\n</transcript>\n Answer in detail in Korean.'.format(transcript)
    }

def make_conversation_message(text):
    message = {
        'role': 'user',
        'content': 'No conversation.'
    }

    if text:
        message['content'] = 'Here is the conversation of the scene in <conversation> tag.\n<conversation>\n{0}\n</conversation>\n'.format(text)

    return message

def chapters_to_vtt(chapters, output_file):
    """
      Constructs a webvtt caption file based on the timestamps from the given chapters.
      Args:
         chapters - the topic points
         output_file - output file where the caption webvtt content is stored.
      Returns:
         None
    """
    vtt_lines = 'WEBVTT\n\n'
    for idx, chapter in enumerate(chapters):
        line = f"{idx}\n{chapter['start']} --> {chapter['end']}\n{chapter['reason']}\n"
        vtt_lines = vtt_lines+line

    util.save_to_file(output_file, vtt_lines)
    return

def inference(model_params):
    model_id = MODEL_ID
    accept = 'application/json'
    content_type = 'application/json'

    bedrock_runtime_client = boto3.client(service_name='bedrock-runtime')

    response = bedrock_runtime_client.invoke_model(
        body=json.dumps(model_params),
        modelId=model_id,
        accept=accept,
        contentType=content_type
    )

    response_body = json.loads(response.get('body').read())

    # patch the json string output with '{' and parse it
    response_content = response_body['content'][0]['text']
    if response_content[0] != '{':
        response_content = '{' + response_content

    try:
        response_content = json.loads(response_content)
    except Exception as e:
        print(colored("Malformed JSON response. Try to repair it...", 'red'))
        try:
            response_content = json_repair.loads(response_content, strict=False)
        except Exception as e:
            print(colored("Failed to repair the JSON response...", 'red'))
            print(colored(response_content, 'red'))
            raise e

    response_body['content'][0]['json'] = response_content
    response_body['model_params'] = model_params

    return response_body


#### 프롬프트 실행 및 출력 검토

모델은 프롬프트 `messages`에서 샘플 출력을 사용하여 지정된 방식으로 포맷된 결과를 반환할 것입니다. 이 경우 비디오의 주제 목록입니다. 각 주제에는 다음이 포함됩니다:

* **start** - HH:MM:SS.MS 형식의 비디오 시작을 기준으로 한 주제의 시작 시간
* **end** - HH:MM:SS.MS 형식의 비디오 시작을 기준으로 한 주제의 종료 시간
* **reason** - `start`와 `end` 사이의 시간 범위에서 대화의 요약

다음 셀을 실행하여 프롬프트를 실행하고 결과로 나온 주제들을 살펴봅니다.

In [ ]:
conversations_response = analyze_conversations(video['transcript'].vtt_file)
video['topics'] = conversations_response['response']['content'][0]['json']['topics']

# show the conversation cost
conversation_cost = brh.display_conversation_cost(conversations_response['response'])

In [ ]:
display(JSON(video['topics'], root='topics', expanded=True))

#### 마지막으로, 모든 매개변수가 채워진 Anthropic Claude에 전달된 실제 프롬프트를 마지막으로 살펴보겠습니다.

`system` 프롬프트는 Anthropic Claude의 작업과 제약 조건을 설명하고, 프롬프트의 `messages` 부분은 FM과의 대화를 모델링합니다.

In [ ]:
JSON(conversations_response['model_params'], root='prompt', expanded=True)

## "topic points" 생성

모델의 출력이 원본 transcript를 정확하게 반영하도록 하기 위해, 출력 JSON은 중복되는 chapter 타임스탬프를 병합하고 chapter 경계를 WebVTT 파일의 실제 캡션 타임스탬프와 정렬하도록 후처리됩니다.

In [ ]:
from lib import topics

# merge overlapped conversation timestamps
video['topics'] = topics.merge_topics(video['topics'])

# validate the conversation timestamps against the caption timestamps
captions = topics.parse_webvtt(video['transcript'].vtt_file)
video['topics'] = topics.validate_timestamps(video['topics'], captions)

# save the conversations
util.save_to_file(os.path.join(video["output_dir"], 'topics.json'), video['topics'])

In [ ]:
JSON(video['topics'], root='topics', expanded=True)

## 주제 시각화
마지막으로, 비디오와 함께 주제를 시각화합니다. chapter 요약을 WebVTT 파일로 출력하고 비디오와 함께 재생할 것입니다.

<div class="alert alert-block alert-info">
참고: 주제들은 12분 전체 비디오에 걸쳐 분포되어 있습니다. 전체 비디오를 재생하는 대신 플레이어 타임라인의 다른 지점을 클릭하여 비디오를 건너뛸 수 있습니다. 5~6개의 주제를 모두 찾을 수 있는지 확인해보세요.
</div>

In [ ]:
video['topics_vtt'] = os.path.join(video['frames'].video_asset_dir(), "topics.vtt")
chapters_to_vtt(video['topics'], video['topics_vtt'])

In [ ]:
# Play the video with topic summaries

from IPython.display import HTML
from base64 import b64encode
start = video['shots'].shots[8]['start_ms']/1000
end = video['shots'].shots[8]['end_ms']/1000
speech_shot_url = f'{video["url"]}#t={start}'

video_html = f"""
<video width="640" height="360" controls>
    <source src="{speech_shot_url}" type="video/mp4">
    <track src="{video['topics_vtt']}" kind="captions" srclang="en" label="English" default>
</video>
"""

HTML(video_html)


방금 generative AI를 사용하여 비디오의 주제 요약을 만들었습니다. 멋진 기술이죠!

🤔 오디오나 비디오 콘텐츠의 다른 시간 세그먼트를 요약하는 다른 사용 사례를 생각할 수 있나요?
<br></br>
🤔 Foundation Model이 찾아내는 주제의 수를 어떻게 늘릴 수 있을까요?


# 워크숍의 나머지 부분에서 사용할 수 있도록 비디오 메타데이터 저장

In [ ]:
%store video




# 다음은 무엇인가요?

이제 기본적인 세그먼테이션을 만들고 비디오 클립에 대한 프롬프트를 작성했으므로, 이러한 기술을 적용하여 몇 가지 사용 사례를 해결할 준비가 되었습니다. 여기서 더 탐구할 사용 사례를 선택할 수 있습니다.

* [Ad break detection and contextual Ad targeting](02-ad-breaks-and-contextual-ad-targeting.ipynb) - 광고 삽입 기회를 식별합니다. 표준 분류법을 사용하여 비디오 콘텐츠를 광고 콘텐츠와 매칭합니다.
* [Video summarization](03-video-summarization.ipynb) - 긴 비디오에서 짧은 형식의 비디오를 생성합니다
* [Semantic video search](04-semantic-video-search.ipynb) - 이미지와 자연어를 사용하여 관련 클립을 찾는 비디오 검색


